# 평가(Evaluation)

## 정확도(Accuracy)

### Titanic 엉터리 분류기

In [1]:
import numpy as np
from sklearn.base import BaseEstimator

#### 타이타닉 데이터 갯수 891 컬럼갯수가 만약 10일때

pred = dt_clf(X_test)
891, 10     train    600, 10
            test     291, 10

In [2]:
""" X = np.ones((10, 5))
X.shape[0] """

' X = np.ones((10, 5))\nX.shape[0] '

In [3]:
""" X = np.zeros((10, 1))
X """

' X = np.zeros((10, 1))\nX '

In [4]:
class MyDummyClassifier(BaseEstimator):
    # fit(), predict() method만 재정의(overriding)
    def fit(self, X, y):#클레스의 메소드는 앞에 self가 들어와야함
        pass
    def predict(self, X): # 만들때 아규먼트 받음 /X는 데이터 프레임으로 들어옴
        pred = np.zeros((X.shape[0], 1)) # shape대신 랭스를 써도 무관
        for i in range(X.shape[0]):
            if X['Sex'].iloc[i] == 0: # 칼럼이 x/ 여성인 경우
                pred[i, 0] = 1 # pred값을 1로 바꿔주고 남성이면 패스
        return pred

#### 전처리를 하지 않고 실행하는 경우

In [5]:
import pandas as pd
titanic_df = pd.read_csv('../00.data/titanic/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

In [7]:
my_clf = MyDummyClassifier()
my_clf.fit(X_train, y_train)
my_pred = my_clf.predict(X_test)

In [8]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, my_pred)

0.659217877094972

### 전처리를 수행한 후 실행하는 경우

In [9]:
from sklearn.preprocessing import LabelEncoder

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [10]:
import pandas as pd

titanic_df = pd.read_csv('../00.data/titanic/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

In [12]:
my_clf = MyDummyClassifier()
my_clf.fit(X_train, y_train)
my_pred = my_clf.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score
print(f'Dummy Classifier의 정확도: {accuracy_score(y_test, my_pred):.4f}')

Dummy Classifier의 정확도: 0.8324


### MyFakeClassifier

### MNIST 손글씨 "Is it seven?"

In [14]:
from sklearn.datasets import load_digits

class MyDummyClassifier(BaseEstimator):
    # fit(), predict() method만 재정의(overriding)
    def fit(self, X, y):#클레스의 메소드는 앞에 self가 들어와야함
        pass

    # 입력값으로 들어오는 X데이터 셋의 크기만큼 모두 0값으로 만들어서 반환
    def predict(self, X): # 만들때 아규먼트 받음 /X는 데이터 프레임으로 들어옴
        return np.zeros((len(X), 1), dtype=bool)


In [15]:
from sklearn.datasets import load_digits

# 사이킷런의 내장 데이터 셋인 load_digits( )를 이용하여 MNIST 데이터 로딩
digits = load_digits()

# digits번호가 7번이면 True 이고 이를 astype(int)로 1로 변환,
# 7번이 아니면 False 이고 0으로 변환.
y = (digits.target == 7).astype(int)
X_train, X_test, y_train, y_test = train_test_split(digits.data, y, random_state=11)

In [16]:
# 불균형한 레이블 데이터 분포도 확인.
print('레이블 테스트 세트 크기 :', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

레이블 테스트 세트 크기 : (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64


In [17]:
# Fake Classifier 로 학습/예습/ 정확도 평가
fakeclf = MyDummyClassifier()
fakeclf.fit(X_train, y_train)
fakepred = fakeclf.predict(X_test)
score = accuracy_score(y_test, fakepred)
print(f'모든 예측을 0으로 하여도 정확도는 :{score:.3f}')


모든 예측을 0으로 하여도 정확도는 :0.900


# 오차 행렬(Confusion Matrix)

### MyFakeClassifier 사례

In [18]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]], dtype=int64)

### 정밀도(Precision)와 재현율(Recall)

In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score #  precision_score(정밀도), recall_score(재현율)

def get_clf_eval(y_test, pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print(f'정확도: {accuracy:.4f}, 정밀도: {precision:.4f}, 재현율: {recall:.4f}, F1 스코어: {f1:.4f}')

In [20]:
titanic_df = pd.read_csv('../00.data/titanic/train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)
X_titanic_df = transform_features(X_titanic_df)
X_train, X_test, y_train, y_test = train_test_split(
    X_titanic_df, y_titanic_df, test_size=0.2, random_state=11
)

### LogisticRegression(로지스틱 회귀) 학습/예측/평가

In [21]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train, y_train)
pred = lr_clf.predict(X_test)
get_clf_eval(y_test, pred)

오차 행렬
[[104  14]
 [ 13  48]]
정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869, F1 스코어: 0.7805


### Precision/Recall Trade-off

In [22]:
pred_proba = lr_clf.predict_proba(X_test) 
pred_proba[:10, :] # 0될 확률 , 1 될 확률

array([[0.4616653 , 0.5383347 ],
       [0.87862763, 0.12137237],
       [0.87727002, 0.12272998],
       [0.88283621, 0.11716379],
       [0.85508952, 0.14491048],
       [0.88231157, 0.11768843],
       [0.88838988, 0.11161012],
       [0.20916926, 0.79083074],
       [0.78258628, 0.21741372],
       [0.36993909, 0.63006091]])

In [23]:
pred[:10]

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1], dtype=int64)

In [24]:
from sklearn.preprocessing import Binarizer

X = [[ 1, -1,  2],
     [ 2,  0,  0],
     [ 0,  1.1, 1.2]]

# threshold 기준값보다 같거나 작으면 0을, 크면 1을 반환
binarizer = Binarizer(threshold=1.1)                     
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [25]:
binarizer = Binarizer(threshold=1)                     
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 1.]]


In [26]:
#Binarizer의 threshold 설정값. 분류 결정 임곗값임.  
custom_threshold = 0.5

# predict_proba( ) 반환값의 두번째 컬럼,
# 즉 Positive 클래스 컬럼 하나만 추출하여 Binarizer를 적용
pred_proba_1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold)
custom_predict = binarizer.fit_transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

오차 행렬
[[104  14]
 [ 13  48]]
정확도: 0.8492, 정밀도: 0.7742, 재현율: 0.7869, F1 스코어: 0.7805


In [27]:
# Binarizer의 threshold 설정값을 0.4로 설정.
custom_threshold = 0.4
pred_proba_1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold)
custom_predict = binarizer.fit_transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

오차 행렬
[[99 19]
 [10 51]]
정확도: 0.8380, 정밀도: 0.7286, 재현율: 0.8361, F1 스코어: 0.7786


In [28]:
# Binarizer의 threshold 설정값을 0.7으로 설정.
custom_threshold = 0.7
pred_proba_1 = pred_proba[:,1].reshape(-1,1)

binarizer = Binarizer(threshold=custom_threshold)
custom_predict = binarizer.fit_transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

오차 행렬
[[116   2]
 [ 28  33]]
정확도: 0.8324, 정밀도: 0.9429, 재현율: 0.5410, F1 스코어: 0.6875


## F1 스코어

In [29]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, pred)
print(f'F1 스코어: {f1:.4f}')

F1 스코어: 0.7805


## ROC AUC 스코어

In [30]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, pred)
print(f'ROC AUC 스코어: {roc_auc:.4f}')

ROC AUC 스코어: 0.8341
